In [ ]:
"如果不担心数据覆盖可以直接运行到底"
import networkx as nx
import os
from tqdm import  tqdm
import pandas as pd


output_folder=r"result\YearStat"
data_folder=r"result\YearOutput"

In [ ]:

node_set=set()
for file in tqdm(os.listdir(data_folder)):
    year,_=file.split(".")
    year_G=nx.read_graphml(os.path.join(data_folder, file))
    node_set.update(year_G.nodes)

100%|██████████| 7/7 [00:01<00:00,  4.36it/s]


In [18]:
node_list=list(node_set)
len(node_list)

17915

In [41]:
# 提取绝对度
all_data = {}
for file in tqdm(os.listdir(data_folder)):
    year, _ = file.split(".")
    year_G = nx.read_graphml(os.path.join(data_folder, file))
    
    edges_to_remove = [edge for edge in year_G.edges 
                       if year_G.edges[edge].get("status", "") == "Stopped"]
    
    year_G.remove_edges_from(edges_to_remove)
    
    # 计算节点度数并存储
    year_degree = dict(year_G.degree())
    all_data[year] = year_degree
    
all_data_table=pd.DataFrame(all_data).fillna(0)
all_data_table["sum"]=all_data_table["2013"]+all_data_table["2014"]+all_data_table["2015"]+all_data_table["2016"]+all_data_table["2017"]+all_data_table["2018"]+all_data_table["2019"]

all_data_table.astype(int).sort_values(by="sum", ascending=False).to_csv(os.path.join(output_folder, "degee2013_2019.csv"),index_label="company")

100%|██████████| 7/7 [00:01<00:00,  3.86it/s]


In [43]:
# 提取相对度
all_data = {}
for file in tqdm(os.listdir(data_folder)):
    year, _ = file.split(".")
    year_G = nx.read_graphml(os.path.join(data_folder, file))
    
    edges_to_remove = [edge for edge in year_G.edges 
                       if year_G.edges[edge].get("status", "") == "Stopped"]
    
    # 删除这些边
    year_G.remove_edges_from(edges_to_remove)
    
    # 计算节点度数并存储
    year_degree = nx.degree_centrality(year_G)
    all_data[year] = year_degree
    
all_data_table=pd.DataFrame(all_data).fillna(0)
all_data_table["average"]=sum(all_data_table[i] for i in all_data_table.columns)/len(all_data_table.columns)

all_data_table.sort_values(by="average", ascending=False).to_csv(os.path.join(output_folder, "degee_centrality2013_2019.csv"),index_label="company")

100%|██████████| 7/7 [00:01<00:00,  4.29it/s]


In [44]:
# 提取临近中心度
all_data = {}
for file in tqdm(os.listdir(data_folder)):
    year, _ = file.split(".")
    year_G = nx.read_graphml(os.path.join(data_folder, file))
    
    edges_to_remove = [edge for edge in year_G.edges 
                       if year_G.edges[edge].get("status", "") == "Stopped"]
    
    # 删除这些边
    year_G.remove_edges_from(edges_to_remove)
    
    # 计算节点度数并存储
    year_degree = nx.closeness_centrality(year_G)
    all_data[year] = year_degree
    
all_data_table=pd.DataFrame(all_data).fillna(0)
all_data_table["average"]=sum(all_data_table[i] for i in all_data_table.columns)/len(all_data_table.columns)

all_data_table.sort_values(by="average", ascending=False).to_csv(os.path.join(output_folder, "closeness_centrality2013_2019.csv"),index_label="company")

100%|██████████| 7/7 [00:02<00:00,  3.04it/s]


In [ ]:
# 提取中介中心度
all_data = {}
for file in tqdm(os.listdir(data_folder)):
    year, _ = file.split(".")
    year_G = nx.read_graphml(os.path.join(data_folder, file))
    
    edges_to_remove = [edge for edge in year_G.edges 
                       if year_G.edges[edge].get("status", "") == "Stopped"]
    
    # 删除这些边
    year_G.remove_edges_from(edges_to_remove)
    
    # 计算节点度数并存储
    year_degree = nx.betweenness_centrality(year_G)
    all_data[year] = year_degree
    
all_data_table=pd.DataFrame(all_data).fillna(0)
all_data_table["average"]=sum(all_data_table[i] for i in all_data_table.columns)/len(all_data_table.columns)

all_data_table.sort_values(by="average", ascending=False).to_csv(os.path.join(output_folder, "betweenness_centrality2013_2019.csv"),index_label="company")

100%|██████████| 7/7 [01:16<00:00, 10.97s/it]


In [ ]:
# 提取Pagerank
all_data = {}
for file in tqdm(os.listdir(data_folder)):
    year, _ = file.split(".")
    year_G = nx.read_graphml(os.path.join(data_folder, file))
    
    edges_to_remove = [edge for edge in year_G.edges 
                       if year_G.edges[edge].get("status", "") == "Stopped"]
    
    # 删除这些边
    year_G.remove_edges_from(edges_to_remove)
    
    # 计算节点度数并存储
    year_degree = nx.pagerank(year_G)
    all_data[year] = year_degree
    
all_data_table=pd.DataFrame(all_data).fillna(0)
all_data_table["average"]=sum(all_data_table[i] for i in all_data_table.columns)/len(all_data_table.columns)

all_data_table.sort_values(by="average", ascending=False).to_csv(os.path.join(output_folder, "pagerank2013_2019.csv"),index_label="company")

100%|██████████| 7/7 [00:03<00:00,  1.76it/s]
